## 使 majority 和其他能夠比較，把訓練資料用成一樣

In [1]:
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *

import pandas as pd
import numpy as np
import os

Using TensorFlow backend.


In [2]:
root_dir = "balance_dataset/Class_3_majority_70_Chang_168"

In [3]:
def map_2_HV_Info(dataframe):
        
        if "multitask_key" not in dataframe.columns: 
                for idx, row in dataframe.iterrows():
                            multitask_key = row["ID"] + "_" + row["side"]   
                            dataframe.loc[idx, "multitask_key"]  = multitask_key 
        
        HV_Dict = {}
        HV_info = pd.read_csv("HV_Table_Clean_20201120.csv")
        
        for idx, row in HV_info.iterrows():
                tooth_num = row["tooth_num"][:-1]
                if not tooth_num.isdigit():
                        self.PBL_Table.drop(idx, inplace=True)
                        continue
                else:

                        if  9 <= int(tooth_num) <= 24: # L--> M, R-->D
                                if "M" in row["tooth_num"]:
                                        row["tooth_num"] = tooth_num + "_L"
                                if "D" in row["tooth_num"]:
                                        row["tooth_num"] = tooth_num + "_R"

                        elif 1 <= int(tooth_num) <= 8 or 24 < int(tooth_num) <= 32:
                                if "M" in row["tooth_num"]:
                                        row["tooth_num"] = tooth_num + "_R"
                                if "D" in row["tooth_num"]:
                                        row["tooth_num"] = tooth_num + "_L"

                        else:
                                print(row)
                                raise ValueError

                multitask_key = row["sub_dir_name"] + "_" + row["image_name"] + "_" + row["tooth_num"]                     
                HV_info.loc[idx,"multitask_key"] = multitask_key
                HV_Dict[multitask_key] = row["Configuration"]

        HV_information = []
        for idx, row in dataframe.iterrows():
                HV_info = HV_Dict.get(row["multitask_key"], "empty")
                if HV_info == "empty": HV_information.append(-999)
                else : HV_information.append(HV_info)
                        
        dataframe["bone_loss"] = HV_information
#         new_HV_data = pd.merge(dataframe, HV_info, on='multitask_key', how='left', suffixes=['_Original', '_HVData'])
        return dataframe

In [4]:
def find_pattern(dataframe): 
        pattern_list = []
        for item in dataframe["Path"]:
                item_split_slash = item.split("/")

                is_clahe = item_split_slash[-2].find("clahe")
                is_Flip  = item_split_slash[-2].find("Flip")

                dir_pattern =  ""
                if   is_clahe == -1 and is_Flip == -1 : dir_pattern = "Ori"
                elif is_clahe != -1 and is_Flip == -1 : dir_pattern = "clahe"
                elif is_clahe == -1 and is_Flip != -1 : dir_pattern = "flip"
                elif is_clahe != -1 and is_Flip != -1 : dir_pattern = "clahe_flip"

                all_pattern = "_".join(item_split_slash[-1].split("_")[1:])
                all_pattern = dir_pattern + "_" + all_pattern 
                pattern_list.append(all_pattern)
        dataframe["pattern"] = pattern_list
        return dataframe

In [5]:
Sheng_ID = ["002555 042513 x", "000408 102419 x", "005627 120209 x", "004151 091409 x",
            "003615 010816 x", "007274 021016 x", "007274 021016 x", "004359 030716 x",
            "001742 082712 x", "002456 060517 x", "1899 120718 x", "004499 110515 x",
            "13529 092513 x",  "000411 112119 x", "008908 090309 x", "003262 103015 x",
            "10689 102418 x", "003670 020718 x", "010953 031618 x", ]        

# Meng 50 + Chang 168 + Sheng 20

In [6]:
directory = [ 
                "Dental_Data/PBL/10_interdental_20200902_Max_4", 
                "Dental_Data/PBL/10_interdental_20200902_Max_4_Flip", 
                "Dental_Data/PBL/10_interdental_clahe_20200902_Max_4", 
                "Dental_Data/PBL/10_interdental_clahe_20200902_Max_4_Flip",
            ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20 

data = load_json(directory, interdental=True)
dataset = json_2_dataframe_PBL_inderdental(data)

Chang_Meng_Sheng = find_pattern(dataset)
Chang_Meng_Sheng = map_2_HV_Info(Chang_Meng_Sheng)
Chang_Meng_Sheng = Chang_Meng_Sheng[ (Chang_Meng_Sheng.State >= 0) & (Chang_Meng_Sheng.bone_loss >= 0) ]

In [7]:
set(Chang_Meng_Sheng.ID) & set(Sheng_ID)

{'000408 102419 x',
 '000411 112119 x',
 '001742 082712 x',
 '002456 060517 x',
 '002555 042513 x',
 '003262 103015 x',
 '003615 010816 x',
 '003670 020718 x',
 '004151 091409 x',
 '004359 030716 x',
 '004499 110515 x',
 '005627 120209 x',
 '007274 021016 x',
 '008908 090309 x',
 '010953 031618 x',
 '10689 102418 x',
 '13529 092513 x',
 '1899 120718 x'}

## Chang 168 + 50 + 20

In [8]:
directory = [ 
                "Dental_Data/PBL/10_interdental_20201110_Max_4", 
                "Dental_Data/PBL/10_interdental_20201110_Max_4_Flip", 
                "Dental_Data/PBL/10_interdental_clahe_20201110_Max_4", 
                "Dental_Data/PBL/10_interdental_clahe_20201110_Max_4_Flip",
            ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20 

data = load_json(directory, interdental=True)
dataset = json_2_dataframe_PBL_inderdental(data)

Chang = find_pattern(dataset)
Chang = map_2_HV_Info(Chang)
Chang = Chang[ (Chang.State >= 0) & (Chang.bone_loss >= 0) ]

In [9]:
Chang

,Path,State,Class,ID,ori_src,source,tooth_num,tooth_type,side,multitask_key,angle,pattern,bone_loss
0,Dental_Data/PBL/10_interdental_20201110_Max_4/...,1,0,000408 102419 x,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,2,NN_191024_151623_BE78A8_6_L,000408 102419 x_NN_191024_151623_BE78A8_6_L,-10,Ori_000408 102419 x_NN_191024_151623_BE78A8_6_...,1
1,Dental_Data/PBL/10_interdental_20201110_Max_4_...,1,0,000408 102419 x,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,2,NN_191024_151623_BE78A8_6_L,000408 102419 x_NN_191024_151623_BE78A8_6_L,-10,flip_000408 102419 x_NN_191024_151623_BE78A8_6...,1
2,Dental_Data/PBL/10_interdental_clahe_20201110_...,1,0,000408 102419 x,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,2,NN_191024_151623_BE78A8_6_L,000408 102419 x_NN_191024_151623_BE78A8_6_L,-10,clahe_000408 102419 x_NN_191024_151623_BE78A8_...,1
3,Dental_Data/PBL/10_interdental_clahe_20201110_...,1,0,000408 102419 x,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,2,NN_191024_151623_BE78A8_6_L,000408 102419 x_NN_191024_151623_BE78A8_6_L,-10,clahe_flip_000408 102419 x_NN_191024_151623_BE...,1
4,Dental_Data/PBL/10_interdental_20201110_Max_4/...,1,0,000408 102419 x,NN_191024_151623_BE78A8,NN_191024_151623_BE78A8_6,6,2,NN_191024_151623_BE78A8_6_R,000408 102419 x_NN_191024_151623_BE78A8_6_R,-10,Ori_000408 102419 x_NN_191024_151623_BE78A8_6_...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
625115,Dental_Data/PBL/10_interdental_clahe_20201110_...,2,1,S594966_2 091718 x,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,3,NN_180917_113933_C0A0B2_26_L,S594966_2 091718 x_NN_180917_113933_C0A0B2_26_L,9,clahe_flip_S594966_2 091718 x_NN_180917_113933...,1
625116,Dental_Data/PBL/10_interdental_20201110_Max_4/...,1,0,S594966_2 091718 x,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,3,NN_180917_113933_C0A0B2_26_R,S594966_2 091718 x_NN_180917_113933_C0A0B2_26_R,9,Ori_S594966_2 091718 x_NN_180917_113933_C0A0B2...,1
625117,Dental_Data/PBL/10_interdental_20201110_Max_4_...,1,0,S594966_2 091718 x,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,3,NN_180917_113933_C0A0B2_26_R,S594966_2 091718 x_NN_180917_113933_C0A0B2_26_R,9,flip_S594966_2 091718 x_NN_180917_113933_C0A0B...,1
625118,Dental_Data/PBL/10_interdental_clahe_20201110_...,1,0,S594966_2 091718 x,NN_180917_113933_C0A0B2,NN_180917_113933_C0A0B2_26,26,3,NN_180917_113933_C0A0B2_26_R,S594966_2 091718 x_NN_180917_113933_C0A0B2_26_R,9,clahe_S594966_2 091718 x_NN_180917_113933_C0A0...,1


In [10]:
set(Chang.ID) & set(Sheng_ID)

{'000408 102419 x',
 '000411 112119 x',
 '001742 082712 x',
 '002456 060517 x',
 '002555 042513 x',
 '003262 103015 x',
 '003615 010816 x',
 '003670 020718 x',
 '004151 091409 x',
 '004359 030716 x',
 '004499 110515 x',
 '005627 120209 x',
 '007274 021016 x',
 '008908 090309 x',
 '010953 031618 x',
 '10689 102418 x',
 '13529 092513 x',
 '1899 120718 x'}

In [11]:
def merge_three_way_table(Meng_Sheng_Chang_df, all_Chang_df, Chang_Majority_df ):
        
        base_columns = ['Path', 'State', 'bone_loss', 'Class', 'ID', 'ori_src', 'source', 'tooth_num', 'tooth_type', 'side', 'angle']
        
        
        Meng_Sheng_Chang_columns = [ i + "_x" for i in base_columns ]
        all_Chang_columns        = [ i + "_y" for i in base_columns ]
        Chang_Majority_columns   = [ i for i in base_columns        ]
        
#         print(len(set(Meng_Sheng_Chang_df.ID) & set(Sheng_ID)))
#         print(len(set(all_Chang_df.ID) & set(Sheng_ID)))
#         print(len(set(Chang_Majority_df.ID) & set(Sheng_ID)))
        
        intersection_table =  pd.merge(Meng_Sheng_Chang_df, all_Chang_df, on='pattern', how='inner')
        intersection_table =  pd.merge(intersection_table, Chang_Majority_df, on='pattern', how='inner')
        
#         print(len(set(intersection_table.ID) & set(Sheng_ID)))
        
        new_Meng_Sheng_Chang = intersection_table[Meng_Sheng_Chang_columns].rename( columns={ new: old for new, old in zip(Meng_Sheng_Chang_columns, base_columns)})
        new_all_Chang =  intersection_table[all_Chang_columns].rename( columns={ new: old for new, old in zip(all_Chang_columns, base_columns)})
        new_majority  =  intersection_table[Chang_Majority_columns].rename(columns= { new: old for new, old in zip(Chang_Majority_columns, base_columns)})
        
        return new_Meng_Sheng_Chang, new_all_Chang, new_majority
        
        
#         all_table = two_table[new_col].rename(columns={  new: old for new, old in zip(new_col , original_col)} )
        
        
        

In [12]:
new_dir = "balance_dataset/Common_Dataset/Multi_Class3"
    
for fold_num in range(1, 6, 1):
        
        if not os.path.isdir(f"{new_dir}_Meng_Sheng_Chang/Fold_{fold_num}"): 
                os.makedirs(f"{new_dir}_Meng_Sheng_Chang/Fold_{fold_num}") 
                
        if not os.path.isdir(f"{new_dir}_All_Chang/Fold_{fold_num}"): 
                os.makedirs(f"{new_dir}_All_Chang/Fold_{fold_num}") 
                
        if not os.path.isdir(f"{new_dir}_Chang_Majority/Fold_{fold_num}"): 
                os.makedirs(f"{new_dir}_Chang_Majority/Fold_{fold_num}") 
                        
        for datasets in ["train_dataset", "valid_dataset", "test_dataset"]:
                majority = pd.read_csv(f"{root_dir}/Fold_{fold_num}/{datasets}.csv")
                majority = find_pattern(majority)
                majority = map_2_HV_Info(majority)
                majority = majority[ (majority.State >= 0) & (majority.bone_loss >= 0) ]
                
                new_Meng_Sheng_Chang, new_all_Chang, new_majority = merge_three_way_table(Chang_Meng_Sheng, Chang, majority)
                
                new_Meng_Sheng_Chang.to_csv(f"{new_dir}_Meng_Sheng_Chang/Fold_{fold_num}/{datasets}.csv", index=False)        
                new_all_Chang.to_csv(f"{new_dir}_All_Chang/Fold_{fold_num}/{datasets}.csv", index=False) 
                new_majority.to_csv(f"{new_dir}_Chang_Majority/Fold_{fold_num}/{datasets}.csv", index=False)
                
                
#                 print(len(new_Meng_Sheng_Chang), len(new_all_Chang), len(new_majority))
                
#         print(merge)
#         original_col = Chang.columns[:-1]
#         new_col = [ i + "_x" for i in original_col ] 
            
#         same_train =  pd.merge(Chang, train_majority, on='pattern', how='inner')
#         same_valid =  pd.merge(Chang, valid_majority, on='pattern', how='inner')
#         same_test  =  pd.merge(Chang, test_majority , on='pattern', how='inner')

#         same_train = same_train[new_col].rename(columns={  new: old for new, old in zip(new_col , original_col)} )
#         same_valid = same_valid[new_col].rename(columns={  new: old for new, old in zip(new_col , original_col)} )
#         same_test  = same_test[new_col].rename(columns={  new: old for new, old in zip(new_col , original_col)} )

#         print(f"======================== Fold {fold_num} ========================")
#         print("Training different"   ,len(train_majority)-len(same_train))
#         print("Validation different" ,len(valid_majority)-len(same_valid))
#         print("Test different"       ,len(test_majority) -len(same_test))
        
#         if not os.path.isdir(f"{new_Chang_dir}/Fold_{fold_num}"): os.makedirs(f"{new_Chang_dir}/Fold_{fold_num}") 
            
#         same_train.to_csv(f"{new_Chang_dir}/Fold_{fold_num}/train_dataset.csv", index=False)
#         same_valid.to_csv(f"{new_Chang_dir}/Fold_{fold_num}/valid_dataset.csv", index=False)
#         same_test.to_csv(f"{new_Chang_dir}/Fold_{fold_num}/test_dataset.csv"  , index=False)